# **LINEAR REGRESSION**

In this exercise, we will run our first example of machine learning model: **a linear regression on a single input variable**. Our exercise comprises two parts.

In **PART 1**, we
1.   import the required Python libraries,
2.   generate some data and plot them,
3.   show different linear regression models,
4.   compute the loss function on the available data,
5.   plot the levels of the loss function.

In **PART 2**, we
1. "learn/train" the linear regression model using OLS,
2. "learn/train" the linear regression model using gradient descent.

**Note** - If you want to modify this notebook, please consider the following Google Colab cheatsheet:
https://colab.research.google.com/github/Tanu-N-Prabhu/Python/blob/master/Cheat_sheet_for_Google_Colab.ipynb#scrollTo=8fFhNQ2EGOcR

We can start.



# **Importing Python libraries**

In [ ]:
# Math library
import numpy as np

# Plotting library
import matplotlib.pyplot as plt

#**PART 1. Introduction to Linear Regression**

In **PART 1** we start with some basic exercises with data, linear regressions and loss functions.

#**Creating some data**

In [ ]:
# Create some input / output data
x = np.array([0.03, 0.19, 0.34, 0.46, 0.78, 0.81, 1.08, 1.18, 1.39, 1.60, 1.65, 1.90])
y = np.array([0.67, 0.85, 1.05, 1.00, 1.40, 1.50, 1.30, 1.54, 1.55, 1.68, 1.73, 1.60])

# Let us print them out
print(x)
print(y)

#**Linear regression**

In [ ]:
# Define 1D linear regression model function
def f(x, beta0, beta1):

  # Linear regression model - computations
  y = beta0 + beta1 * x

  return y

In [ ]:
# Function to help plot the data
def plot(x, y, beta0, beta1):

    fig,ax = plt.subplots()
    ax.scatter(x,y)
    plt.xlim([0,2.0])
    plt.ylim([0,2.0])
    ax.set_xlabel('Input x')
    ax.set_ylabel('Output y')

    # Draw line on the plot
    x_line = np.arange(0,2,0.01)
    y_line = f(x_line, beta0, beta1)
    plt.plot(x_line, y_line,'b-',lw=2)

    plt.show()

In [ ]:
# Let us play around with the linear regression

# Set the intercept and slope - examples from the slides: (beta0, beta1) = (0.0, 1.0), (1.0, -0.4), (1.2, -0.1)
beta0 = 0.83
beta1 = 0.52

# Plot the data and the model
plot(x, y, beta0, beta1)

#**Loss function**

In [ ]:
# Function to calculate the loss
def compute_loss(x, y, beta0, beta1):

  # We compute the loss in two steps.
  # Step 1: I start computing the squared differences, per each data point
  loss = (f(x, beta0, beta1)-y)**2

  # Step 2: I sum the squared differences
  loss = np.sum(loss)

  return loss

In [ ]:
# Compute the loss for our current model
loss = compute_loss(x,y, beta0, beta1)
print(f'Your Loss = {loss:3.2f}')

In [ ]:
# Let us visualize the levels of the loss function using a heatmap

# Make a 2D grid of possible phi0 and phi1 values
beta0_mesh, beta1_mesh = np.meshgrid(np.arange(0.0,2.0,0.02), np.arange(-1.0,1.0,0.02))

# Make a 2D array for the losses
all_losses = np.zeros_like(beta1_mesh)

# Run through each 2D combination of beta0, beta1 and compute loss
for indices,temp in np.ndenumerate(beta1_mesh):
    all_losses[indices] = compute_loss(x, y, beta0_mesh[indices], beta1_mesh[indices])

In [ ]:
# Plot the loss function as a heatmap
fig = plt.figure()
ax = plt.axes()
fig.set_size_inches(7,7)
levels = 256
ax.contourf(beta0_mesh, beta1_mesh, all_losses ,levels)
levels = 40
ax.contour(beta0_mesh, beta1_mesh, all_losses ,levels, colors=['#80808080'])
ax.set_ylim([1,-1])
ax.set_xlabel('Intercept')
ax.set_ylabel('Slope')

# Plot the position of the beta's of the linear regression model on the loss function
# The best model should have beta's close to where the loss reaches its minimum
ax.plot(beta0, beta1, 'ro')
plt.show()

#**PART 2. Learning/Training a Linear Regression (March 15th)**

We will consider this second part of the Colab on March 15th, after having discussed gradient descent.

## **Method 1: OLS using the sklearn function LinearRegression**
With this method, all we need is to call the `sklearn` `LinearRegression()` function and specify data in the appropriate shape.

The function uses OLS to compute the estimation of the intercept `beta0` and slope `beta1`.

In [ ]:
# Let us import the linear regression function from sklearn
from sklearn.linear_model import LinearRegression

In [ ]:
# Learning/training the linear regression on our data (no train vs. test split this time for simplicity)
lin_reg = LinearRegression().fit(x.reshape(-1, 1), y.reshape(-1, 1)) # we need to reshape inputs

print('Estimated intercept:', lin_reg.intercept_.round(2)) # let us keep only two decimal places (you can do it in different ways)
print('Estimated slope:', lin_reg.coef_.round(2)) # let us keep only two decimal places (you can do it in different ways)

In [ ]:
# the learned/trained model 'lin_reg' computes predictions on new data points (we need to shape them correctly, however - arrays of shape (1, 1))
lin_reg.predict(np.array([[0.15]]))

## **Method 2: Gradient Descent (by hand)**
With this method, we will define a gradient descent function by hand and use it to compute the estimates of the intercept `beta0` and the slope `beta1`.

Key here is the analytical computations of the gradient of the loss function $L(\boldsymbol{\theta}|x_i,y_i)$ at each point $(x_i,y_i)$ - that is each element of the arrays `x` and `y`.
We have:

\begin{eqnarray}
\frac{\partial L(\boldsymbol{\theta}|x_i,y_i)}{\partial\theta_0} &=& -2(y_i-\theta_0-\theta_1 x_i), \\
\frac{\partial L(\boldsymbol{\theta}|x_i,y_i)}{\partial\theta_1} &=& x_i \frac{\partial L(\boldsymbol{\theta}|x_i,y_i)}{\partial\theta_0}.
\end{eqnarray}

We can start computing the gradients and implement the gradient descent algorithm by hand.

In [ ]:
# gradient descent function
def gradient_descent_func(x, y, beta0_start, beta1_start, eta, t_max):

    # initialization: we are at step 0
    t=0

    # initialization: specify the starting values for the beta's
    beta0 = beta0_start
    beta1 = beta1_start

    # we implement the gradient descent algorithm update rule
    while t < t_max:

      # gradient of the loss wrt beta's for each element of the arrays x, y
      pder_loss_beta0 = -2 * (y-beta0-beta1 * x)
      pder_loss_beta1 = x * pder_loss_beta0

      # gradient descent: updating beta's
      beta0 = beta0 - eta * np.mean(pder_loss_beta0)
      beta1 = beta1 - eta * np.mean(pder_loss_beta1)

      # increment step by one
      t=t+1

    # when t_max is reached exit giving the beta's at t_max
    return beta0, beta1

In [ ]:
# Let us estimate beta's using our gradient descent function
# Good choices: (0.10, 0.10, 0.10, 100), (1, 1, 0.10, 100)
# Hint: Try (0.10, 0.10, 1, x)...see what happens incrementing x one by one...

beta0_start=1
beta1_start=1
eta=0.1
t_max=100

gradient_descent_func(x, y, beta0_start, beta1_start, eta, t_max)